In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from collections import deque

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [7]:
# Actions that we try to detect
actions = np.array(['Hi', 'I am', 'From', 'Singapore', 'Paiseh', 'I', 'Cannot', 'Speak', 'But', 'Machine', 'Learning', 'Can', 'Help', 'Translate'])

In [8]:
label_map = {label:num for num, label in enumerate(actions)}

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout


In [10]:
model = Sequential() # changed activation to tanh
model.add(LSTM(64, return_sequences=True, activation='tanh', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='tanh'))
model.add(LSTM(64, return_sequences=False, activation='tanh'))
model.add(Dense(64, activation='tanh'))
model.add(Dense(32, activation='tanh'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [11]:
model.load_weights('lstm_model___Date_Time_2022_04_04__20_36_33___Loss_0.22720035910606384___Accuracy_0.9785714149475098.h5')

In [21]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.8

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 10: 
                sentence = sentence[-10:]

            # Viz probabilities
            #image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,440), (640, 480), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,460), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Speak
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Speak
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Speak
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Speak
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Speak
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Speak
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Speak
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Speak
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Speak
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Speak
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Speak
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Speak
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Speak
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Speak
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Speak
<class 'mediapipe.python.solution_base.SolutionOutputs'

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Singapore
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Singapore
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Singapore
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Singapore
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Singapore
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Singapore
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Singapore
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Singapore
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Singapore
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Singapore
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Translate
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Translate
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Translate
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Translate
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Trans

<class 'mediapipe.python.solution_base.SolutionOutputs'>
But
<class 'mediapipe.python.solution_base.SolutionOutputs'>
But
<class 'mediapipe.python.solution_base.SolutionOutputs'>
But
<class 'mediapipe.python.solution_base.SolutionOutputs'>
But
<class 'mediapipe.python.solution_base.SolutionOutputs'>
But
<class 'mediapipe.python.solution_base.SolutionOutputs'>
But
<class 'mediapipe.python.solution_base.SolutionOutputs'>
But
<class 'mediapipe.python.solution_base.SolutionOutputs'>
But
<class 'mediapipe.python.solution_base.SolutionOutputs'>
But
<class 'mediapipe.python.solution_base.SolutionOutputs'>
But
<class 'mediapipe.python.solution_base.SolutionOutputs'>
But
<class 'mediapipe.python.solution_base.SolutionOutputs'>
But
<class 'mediapipe.python.solution_base.SolutionOutputs'>
But
<class 'mediapipe.python.solution_base.SolutionOutputs'>
But
<class 'mediapipe.python.solution_base.SolutionOutputs'>
But
<class 'mediapipe.python.solution_base.SolutionOutputs'>
But
<class 'mediapipe.python

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Cannot
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Help
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Help
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Help
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Help
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Help
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Speak
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Speak
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Speak
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Speak
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Speak
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Speak
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Speak
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Speak
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Speak
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ca

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Learning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Learning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Learning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Learning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Learning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Learning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Learning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Learning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Learning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Help
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Help
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Help
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Help
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Can
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Can
<class 'mediapipe.python.solution_ba

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Translate
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Translate
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Translate
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Translate
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Translate
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Translate
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Translate
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Translate
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Translate
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Translate
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Translate
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Translate
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Translate
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Translate
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Trans